## 4.2.	실험 설계와 평가 기준
본 연구에서는 RAG와 RAPTOR의 성능을 효과적으로 비교하기 위해, 실험 기준 4단계를 사용자의 구체적인 질문 유형인 상황과 사용자의 추상적인 질문 유형인 상황으로 나누어 평가하였다.

### 4.2.1.	실험 설계 개요
실험은 두 가지 질문 유형에 대해 진행되었다.
-	구체적인 질문: 특정 기사에 대한 사용자의 질문으로, 사용자가 정확한 정보를 검색하는 경우를 의미한다. 구체적인 질문에 있어서의 평가는 모델의 정확도와 응답 시간을 비교하여 각 기법의 성능을 평가하였다.
-	추상적인 질문: 특정 주제, 일간 이슈, 주간 이슈에 대한 사용자의 질문으로, 사용자가 보다 일반적이고 포괄적인 정보를 묻는 경우를 의미한다. 추상적인 질문에 있어서의 평가는 요약의 정확도와 응답 시간을 비교하여 각 기법의 성능을 평가하였다.

### 구체적인 질문 측정: 특정 기사

In [1]:
import pandas as pd

qa_data = pd.read_excel('./Specific_News_QA_data_set.xlsx')
qa_data.columns

Index(['질문 유형', '일자', 'Question', 'Answer', 'LLM Answer', 'RAG Answer',
       'RAPTOR Answer', 'LLM Score', 'RAG Score', 'RAPTOR Score', 'News 제목',
       'News URL'],
      dtype='object')

BERTScore 측정

In [ ]:
from bert_score import score

# BERTScore 기반 F1 Score 계산 함수 정의
def calculate_bert_score(reference, candidate, lang="ko"):
    """
    BERTScore를 기반으로 참조 문장(reference)과 후보 문장(candidate)의 F1 Score를 계산합니다.
    
    Args:
        reference (str): 참조 문장
        candidate (str): 후보 문장
        lang (str): BERTScore에서 사용할 언어 (기본값: "ko" - 한국어)
        
    Returns:
        float: BERTScore 기반 F1 Score
    """
    # BERTScore 계산
    P, R, F1 = score([candidate], [reference], lang=lang, verbose=False)
    
    # F1.mean() 값을 반환
    return F1.mean().item()

In [4]:
# LLM Score, RAG Score, RAPTOR Score 계산
qa_data['LLM Score'] = qa_data.apply(lambda x: calculate_bert_score(x['Answer'], x['LLM Answer']), axis=1)
qa_data['RAG Score'] = qa_data.apply(lambda x: calculate_bert_score(x['Answer'], x['RAG Answer']), axis=1)
qa_data['RAPTOR Score'] = qa_data.apply(lambda x: calculate_bert_score(x['Answer'], x['RAPTOR Answer']), axis=1)

In [5]:
qa_data

,질문 유형,일자,Question,Answer,LLM Answer,RAG Answer,RAPTOR Answer,LLM Score,RAG Score,RAPTOR Score,News 제목,News URL
0,구체적 유형 질문,20240826,우리은행은 언제 손태승 전 회장의 친인척과 관련된 부당대출을 인지했나요?,"우리은행은 2023년 4분기 중 감사에서 해당 대출이 문제가 된 것을 인지했으며, ...",우리은행 부당대출 사건 개요\n사건 배경: 우리은행은 손태승 전 회장의 친인척과 관...,우리는 은행은 작년 9월에서 10월 사이에 손태승 전 회장의 친인척과 관련된 부당대...,우리은행은 손태승 전 회장의 친인척과 관련된 부당대출을 작년 9월에서 10월 사이에...,0.639751,0.755150,0.758593,"금감원 ""우리은행 부당대출 대응 미흡···내부통제 매우 심각""",https://www.ajunews.com/view/20240825105852306
1,구체적 유형 질문,20240826,현대차 전기차 모델들이 세계 주요 충돌안전평가에서 어떤 성적을 받았나요?,현대차그룹의 E-GMP 기반 전기차 모델들이 미국과 유럽 등 세계 주요 충돌안전평가...,현대차 전기차 모델 충돌안전평가 성적\n현대차 전기차 모델들이 최근 세계 주요 충돌...,현대차 전기차 모델의 충돌안전평가 성적\n현대차 그룹의 전기차 모델들은 세계 주요 ...,현대차 전기차 모델들의 충돌안전평가 성적\n유럽 및 미국 충돌 안전 평가 성적:\n...,0.708379,0.772976,0.808471,"현대차 전기차, 美·유럽 안전평가 최고",https://www.hankyung.com/article/2024082542181
2,구체적 유형 질문,20240827,삼성 준법감시위원회가 회비 납부 전에 어떤 조건을 제시했나요?,"준감위는 지난해 8월 발표한 '한경협 가입 권고안'에 따라, 삼성 계열사들이 회비를...","삼성 준법감시위원회의 조건\n투명성 강화: 회비 납부 전, 삼성 준법감시위원회는 투...",삼성 준법감시위원회의 조건\n삼성 준법감시위원회(준감위)는 삼성 계열사의 한국경제인...,삼성 준법감시위원회의 조건\n삼성 준법감시위원회(준감위)는 삼성 계열사의 한국경제인...,0.626001,0.736922,0.717948,"삼성 준감위, 한경협 회비 납부 승인",https://www.hankyung.com/article/2024082671071
3,구체적 유형 질문,20240827,명절 연휴 동안 응급실에서 발생할 수 있는 주요 문제는 무엇인가요?,"명절 연휴 동안 응급실은 환자 수가 평소의 두 배로 증가하고, 전공의의 수가 부족해...",명절 연휴 응급실 주요 문제\n환자 급증: 명절 기간 동안 응급실 방문자가 급증하여...,명절 연휴 동안 응급실에서 발생할 수 있는 주요 문제\n환자 급증: 명절 연휴 동안...,명절 연휴 동안 응급실에서 발생할 수 있는 주요 문제\n응급 의료 위기 상황: 명절...,0.673603,0.713110,0.760052,붕괴 직전 응급실… “추석 의료대란 현실화”,https://www.segye.com/newsView/20240826514507
4,구체적 유형 질문,20240828,SK이노베이션과 SK E&S의 합병이 성공적으로 통과된 비율은 얼마인가요?,SK이노베이션과 SK E&S의 합병안은 86%의 찬성률로 임시 주주총회를 통과했습니다.,SK이노베이션과 SK E&S 합병\n합병 성공률: SK이노베이션과 SK E&S의 합...,SK이노베이션과 SK E&S의 합병\n합병 승인 비율: SK이노베이션과 SK E&S...,SK이노베이션과 SK E&S의 합병은 주주총회에서 **85.76%**의 찬성률로 성...,0.637527,0.758545,0.893750,SK이노·SK E&S 합병안 찬성률 86%로 통과,https://www.segye.com/newsView/20240827516108
5,구체적 유형 질문,20240828,간호법 제정안의 본회의 통과를 위한 논의가 언제 열릴 예정인가요?,간호법 제정안의 본회의 통과를 위한 논의는 28일 국회 본회의 전에 보건복지위원회 ...,간호법 제정안 논의 일정\n간호법 제정안은 최근 주요 이슈로 떠오르고 있습니다.\n...,간호법 제정안 본회의 통과 일정\n간호법 제정안은 8월 28일 오후에 예정된 본회의...,간호법 제정안 본회의 통과 일정\n간호법 제정안은 8월 28일 오후에 예정된 본회의...,0.657483,0.764084,0.799102,"복지위, 28일 본회의 전 간호법 막판 논의 추진",www.kado.net/news/articleView.html?idxno=1262310
6,구체적 유형 질문,20240829,보건의료노조의 총파업이 철회된 이유는 무엇인가요?,"보건의료노조의 총파업은 고려대, 이화여대, 중앙대 의료원 등과의 교섭이 성공적으로 ...",보건의료노조 총파업 철회 이유\n정부와의 협상 타결: 보건의료노조는 정부와의 협상에...,보건의료노조 총파업 철회 이유\n보건의료노조의 총파업이 철회된 이유는 대부분의 병원...,보건의료노조 총파업 철회 이유\n보건의료노조의 총파업이 철회된 이유는 여러 병원과의...,0.654615,0.786277,0.778754,고려대 이화여대 병원 교섭 타결...보건의료노조 파업 철회 가능성,https://www.dt.co.kr/contents.html?article_no=...
7,구체적 유형 질문,20240829,현대차가 전기차 시장 수요 감소에 대응하기 위해 발표한 중장기 전략은 무엇인가요?,현대차는 전기차 시장의 일시적 수요 정체(캐즘)에 대응하기 위해 하이브리드 차종을 ...,현대차의 중장기 전략\n전기차 모델 확대: 현대차는 다양한 전기차 모델을 출시하여 ...,현대차의 중장기 전략\n현대차는 전기차 시장 수요 감소에 대응하기 위해 하이브리드 ...,현대차는 전기차 시장의 일시적 수요 감소에 대응하기 위해 하이브리드 차량에 중점을 ...,0.635142,0.732276,0.797254,현대차 “하이브리드로 ‘캐즘’ 극복”… 제네시스 신차 내놓는다,https://www.donga.com/news/Economy/article/all...
8,구체적 유형 질문,20240830,윤석열 대통령이 제시한 국민연금 개혁의 3대 원칙은 무엇인가요?,"윤석열 대통령은 국민연금 개혁의 3대 원칙으로 지속가능성, 세대 간 공정성, 노후소...",윤석열 대통령의 국민연금 개혁 3대 원칙\n지속 가능성 강화\n국민연금의 재정적 지...,윤석열 대통령이 제시한 국민연금 개혁의 3대 원칙\n지속 가능성: 장기적으로 지속 ...,윤석열 대통령이 제시한 국민연금 개혁의 3대 원칙\n지속 가능성: 연금 제도의 장기...,0.676402,0.786422,0.735102,"국가 지급 보장' 법문화…""청년세대 수긍할 개혁""",https://news.sbs.co.kr/news/endPage.do?news_id...
9,구체적 유형 질문,20240830,국민의힘과 정부가 추진하고 있는 딥페이크 성범죄 처벌 강화 방안은 무엇인가요?,국민의힘과 정부는 딥페이크 성범죄의 처벌 수위를 현행 최대 징역 5년에서 최대 징역...,딥페이크 성범죄 처벌 강화 방안\n법률 개정: 국민의힘과 정부는 딥페이크 성범죄에 ...,딥페이크 성범죄 처벌 강화 방안\n처벌 기준 강화: 허위 영상물 편집 및 반포 행위...,딥페이크 성범죄 처벌 강화 방안\n처벌 수위 강화: 딥페이크 성범죄에 대한 처벌을 ...,0.666643,0.702999,0.763428,"국민의힘-정부, 딥페이크 성범죄에 최대 징역 7년 추진…""촉법 소년 기준도 낮춰야""",https://www.yeongnam.com/web/view.php?key=2024...


In [6]:
print(f"LLM 응답 F1-Score: {qa_data['LLM Score'].mean()}")
print(f"RAG 응답 F1-Score: {qa_data['RAG Score'].mean()}")
print(f"RAPTOR 응답 F1-Score: {qa_data['RAPTOR Score'].mean()}")

LLM 응답 F1-Score: 0.6575544059276581
RAG 응답 F1-Score: 0.7508760333061218
RAPTOR 응답 F1-Score: 0.7812452614307404


In [7]:
print("=== LLM 응답 Score ===")
qa_data['LLM Score'].describe()

=== LLM 응답 Score ===


count    10.000000
mean      0.657554
std       0.024728
min       0.626001
25%       0.638083
50%       0.656049
75%       0.671863
max       0.708379
Name: LLM Score, dtype: float64

In [8]:
print("=== RAG 응답 Score ===")
qa_data['RAG Score'].describe()

=== RAG 응답 Score ===


count    10.000000
mean      0.750876
std       0.028898
min       0.702999
25%       0.733437
50%       0.756847
75%       0.770753
max       0.786422
Name: RAG Score, dtype: float64

In [9]:
print("=== RAPTOR 응답 Score ===")
qa_data['RAPTOR Score'].describe()

=== RAPTOR 응답 Score ===


count    10.000000
mean      0.781245
std       0.048757
min       0.717948
25%       0.758958
50%       0.771091
75%       0.798640
max       0.893750
Name: RAPTOR Score, dtype: float64

### 추상적인 질문 측정: 특정 주제

In [10]:
qa_data = pd.read_excel('./Specific_Topic_QA_data_set.xlsx')
qa_data.columns

Index(['질문 유형', '일자', 'Question', 'Answer', 'LLM Answer', 'RAG Answer',
       'RAPTOR Answer', 'LLM Score', 'RAG Score', 'RAPTOR Score'],
      dtype='object')

In [11]:
# LLM Score, RAG Score, RAPTOR Score 계산
qa_data['LLM Score'] = qa_data.apply(lambda x: calculate_bert_score(x['Answer'], x['LLM Answer']), axis=1)
qa_data['RAG Score'] = qa_data.apply(lambda x: calculate_bert_score(x['Answer'], x['RAG Answer']), axis=1)
qa_data['RAPTOR Score'] = qa_data.apply(lambda x: calculate_bert_score(x['Answer'], x['RAPTOR Answer']), axis=1)

In [12]:
qa_data

,질문 유형,일자,Question,Answer,LLM Answer,RAG Answer,RAPTOR Answer,LLM Score,RAG Score,RAPTOR Score
0,추상적 유형 질문,20240829,현대차 10년간 120조 투자 계획에 대해서 종합적인 요약본을 제공해주세요.,현대자동차는 전기차 시장의 수요 정체(캐즘)를 극복하고 미래 모빌리티 시장에서 글로...,현대차 10년간 120조 투자 계획 요약\n투자 규모: 현대차는 향후 10년간 12...,현대차의 10년간 120조 투자 계획 요약\n투자 규모: 현대차는 향후 10년간 총...,현대차 10년간 120조 투자 계획 요약\n현대자동차는 향후 10년간 총 120조 ...,0.679091,0.678797,0.737444


### 추상적인 질문 측정: 일간 이슈

In [13]:
qa_data = pd.read_excel('./Day_Issue_QA_data_set.xlsx')
qa_data.columns

Index(['질문 유형', '일자', 'Question', 'Answer', 'LLM Answer', 'RAG Answer',
       'RAPTOR Answer', 'LLM Score', 'RAG Score', 'RAPTOR Score'],
      dtype='object')

In [14]:
# LLM Score, RAG Score, RAPTOR Score 계산
qa_data['LLM Score'] = qa_data.apply(lambda x: calculate_bert_score(x['Answer'], x['LLM Answer']), axis=1)
qa_data['RAG Score'] = qa_data.apply(lambda x: calculate_bert_score(x['Answer'], x['RAG Answer']), axis=1)
qa_data['RAPTOR Score'] = qa_data.apply(lambda x: calculate_bert_score(x['Answer'], x['RAPTOR Answer']), axis=1)

In [15]:
qa_data

,질문 유형,일자,Question,Answer,LLM Answer,RAG Answer,RAPTOR Answer,LLM Score,RAG Score,RAPTOR Score
0,추상적 유형 질문,20240826,2024년 08월 26일 주요 기사를 알려주세요.,\n2024년 8월 26일의 주요 이슈는 다음과 같습니다:\n\n헤즈볼라와 하마스의...,"2024년 08월 26일 주요 기사 요약\n정치:\n대통령, 새로운 경제 정책 발표...",2024년 8월 26일 주요 기사 요약\n태풍 산산: '사상 최강' 태풍 산산이 일...,2024년 8월 26일 주요 기사 요약\n현대자동차 그룹의 전기차 성과\n현대차의 ...,0.608381,0.652723,0.681656
1,추상적 유형 질문,20240827,2024년 08월 27일 주요 기사를 알려주세요.,2024년 8월 27일의 주요 이슈는 다음과 같습니다:\n\n김문수의 청문회 발언 ...,"2024년 08월 27일 주요 기사 요약\n정치:\n대통령, 경제 회복을 위한 새로...",2024년 08월 27일 주요 기사\n태풍 산산 일본 상륙: 강력한 태풍 산산이 일...,2024년 08월 27일 주요 기사\n현대자동차 그룹의 전기차 성과\n현대차의 E-...,0.616098,0.654261,0.703219
2,추상적 유형 질문,20240828,2024년 08월 28일 주요 기사를 알려주세요.,2024년 8월 28일의 주요 이슈는 다음과 같습니다:\n\n내년 예산안 발표: 정...,2024년 08월 28일 주요 기사 요약\n정치\n대통령 연설: 대통령은 경제 회복...,2024년 8월 28일 주요 기사\n태풍 산산 일본 상륙: 강력한 태풍 산산이 일본...,2024년 08월 28일 주요 기사\n현대자동차 전기차 성과\n현대차의 전기차 모델...,0.716176,0.684870,0.680529
3,추상적 유형 질문,20240829,2024년 08월 29일 주요 기사를 알려주세요.,\n2024년 8월 29일의 주요 이슈는 다음과 같습니다:\n\n딥페이크 성범죄 대...,2024년 08월 29일 주요 기사 요약\n경제\n한국 경제 성장률: 한국의 경제 ...,2024년 8월 29일 주요 기사\n경기도 부천시 화재 사건: 경기도 부천시의 한 ...,2024년 8월 29일 주요 기사 요약\n현대자동차 그룹의 전기차 성과\n현대차의 ...,0.704722,0.696255,0.675291
4,추상적 유형 질문,20240830,2024년 08월 30일 주요 기사를 알려주세요.,\n2024년 8월 30일의 주요 이슈는 다음과 같습니다:\n\n허위 영상물 처벌 ...,2024년 08월 30일 주요 기사 요약\n경제\n한국 경제 성장률: 2분기 GDP...,2024년 8월 30일 주요 기사 요약\n태풍 산산 일본 상륙: 강력한 태풍 산산이...,2024년 8월 30일 주요 기사 요약\n현대자동차 그룹의 전기차 성과\n현대차의 ...,0.682479,0.659789,0.665375


In [16]:
print(f"LLM 응답 F1-Score: {qa_data['LLM Score'].mean()}")
print(f"RAG 응답 F1-Score: {qa_data['RAG Score'].mean()}")
print(f"RAPTOR 응답 F1-Score: {qa_data['RAPTOR Score'].mean()}")

LLM 응답 F1-Score: 0.6655710935592651
RAG 응답 F1-Score: 0.6695794463157654
RAPTOR 응답 F1-Score: 0.6812138080596923


### 추상적인 질문 측정: 주간 이슈

In [17]:
qa_data = pd.read_excel('./Week_Issue_QA_data_set.xlsx')
qa_data.columns

Index(['질문 유형', 'Question', 'Answer', 'LLM Answer', 'RAG Answer',
       'RAPTOR Answer', 'LLM Score', 'RAG Score', 'RAPTOR Score'],
      dtype='object')

In [18]:
# LLM Score, RAG Score, RAPTOR Score 계산
qa_data['LLM Score'] = qa_data.apply(lambda x: calculate_bert_score(x['Answer'], x['LLM Answer']), axis=1)
qa_data['RAG Score'] = qa_data.apply(lambda x: calculate_bert_score(x['Answer'], x['RAG Answer']), axis=1)
qa_data['RAPTOR Score'] = qa_data.apply(lambda x: calculate_bert_score(x['Answer'], x['RAPTOR Answer']), axis=1)

In [19]:
qa_data

,질문 유형,Question,Answer,LLM Answer,RAG Answer,RAPTOR Answer,LLM Score,RAG Score,RAPTOR Score
0,추상적 유형 질문,2024년 08월 5주차 주간 이슈 기사를 종합하여 요약해주세요.,2024년 8월 26일부터 30일까지의 주요 뉴스는 다음과 같습니다:\n\n**20...,2024년 08월 5주차 주간 이슈 요약\n주요 뉴스\n정치\n대통령의 해외 순방 ...,2024년 8월 5주차 주간 이슈 요약\n정치: 이번 주 국정 브리핑에서는 '4+1...,2024년 8월 5주차 주간 이슈 요약\n현대자동차 전기차 성과: 현대차의 전기차 ...,0.62356,0.677042,0.73856


In [20]:
print(f"LLM 응답 F1-Score: {qa_data['LLM Score'].mean()}")
print(f"RAG 응답 F1-Score: {qa_data['RAG Score'].mean()}")
print(f"RAPTOR 응답 F1-Score: {qa_data['RAPTOR Score'].mean()}")

LLM 응답 F1-Score: 0.6235597133636475
RAG 응답 F1-Score: 0.6770419478416443
RAPTOR 응답 F1-Score: 0.7385600805282593
